In [36]:
from llama_stack_client import LlamaStackClient, Agent
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich
import os

namespace = "oom-test"
# Uncomment to run on a hosted Llama Stack server
client = LlamaStackClient(base_url="http://localhost:8321")

MODEL_ID = "llama-4-scout-17b-16e-w4a16"

base_agent_config = dict(
    model=MODEL_ID,
    instructions="You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.",
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    tools=[
        {
            "name": "mcp::openshift",
            "args": {"namespace": namespace}
        }
    ],
)

vanilla_agent_config = {
    **base_agent_config,
    "instructions": """
    You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.
    """,
}

vanilla_agent = Agent(client, **vanilla_agent_config)
prompt_chaining_session_id = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid.uuid4()}")

prompts = [
    """
    Analyze the namespace '{namespace}' by:
            1. Listing all pods 
            2. get pod logs for the pods listed above
            2. Checking events for any errors or warnings
            3. Providing initial assessment""",
    """Format the output as:
            - Namespace: {namespace}
            - Pod: [pod name]
            - Error Title: [error type/reason] 
            - Error Timestamp: [timestamp]
            - Error Description: [detailed error message]
            If no errors found, return: "No error found""",
]

for i, prompt in enumerate(prompts):    
    response = vanilla_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=prompt_chaining_session_id,
        stream=False,
    )
    print("************ Turn: ", i, "****************")
    print(response.output_message.content)
    print("\n")

************ Turn:  0 ****************

************ Turn:  1 ****************

- Namespace: oom-test
- Pod: oom-test-deployment-55756fd6d7-s8d2k
- Error Title: [OOMKilled]
- Error Timestamp: [2025-10-04T02:44:20Z]
- Error Description: [Container oom-test-container terminated due to OOM]

In [38]:
from llama_stack_client import LlamaStackClient, Agent
from rich.pretty import pprint
import json
import uuid
from pydantic import BaseModel
import rich
import os

namespace = "oom-test"
# Uncomment to run on a hosted Llama Stack server
client = LlamaStackClient(base_url="http://localhost:8321")

MODEL_ID = "llama-4-scout-17b-16e-w4a16" #"llama-3-2-3b" #"r1-qwen-14b-w4a16" # "llama-4-scout-17b-16e-w4a16"

base_agent_config = dict(
    model=MODEL_ID,
    instructions="You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.",
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    tools=[
        {
            "name": "mcp::openshift",
            "args": {"namespace": namespace}
        }
    ],
)

vanilla_agent_config = {
    **base_agent_config,
    "instructions": """
    You are an expert OpenShift administrator. Analyze the namespace '{namespace}' for errors and issues.
    """,
}

vanilla_agent = Agent(client, **vanilla_agent_config)
prompt_chaining_session_id = vanilla_agent.create_session(session_name=f"vanilla_agent_{uuid.uuid4()}")

prompts = [
    """
    Analyze the namespace '{namespace}' by:
            1. Listing all pods 
            2. get pod logs for the pods listed above
            2. Checking events for any errors or warnings
            3. Providing initial assessment""",
    """Format the output as JSON:
            {
                "namespace": "{namespace}",
                "analysis_results": [
                    {
                        "Pod": "[pod name]",
                        "Error Title": "[error type/reason]",
                        "Error Timestamp": "[timestamp]"
                        "Error Description": "[detailed error message]"
                    }
                ]
            }
            """,
]

for i, prompt in enumerate(prompts):    
    response = vanilla_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=prompt_chaining_session_id,
        stream=False,
    )
    print("************ Turn: ", i, "****************")
    print(response.output_message.content)
    print("\n")

************ Turn:  0 ****************

************ Turn:  1 ****************

Here is a JSON output of the analysis:

```json
{
  "namespace": "oom-test",
  "analysis_results": [
    {
      "Pod": "oom-test-deployment-55756fd6d7-s8d2k",
      "Error Title": "OOMKilled",
      "Error Timestamp": "2025-10-04T02:49:41Z",
      "Error Description": "The container oom-test-container was terminated due to OOM (Out of Memory) and was 
restarted multiple times."
    },
    {
      "Pod": "oom-test-deployment-55756fd6d7-s8d2k",
      "Error Title": "CrashLoopBackOff",
      "Error Timestamp": "2025-10-04T02:49:41Z",
      "Error Description": "The pod oom-test-deployment-55756fd6d7-s8d2k is experiencing a CrashLoopBackOff due to 
the OOMKilled error."
    }
  ]
}
```

Initial Assessment:

The pod `oom-test-deployment-55756fd6d7-s8d2k` in the `oom-test` namespace is experiencing OOM (Out of Memory) 
issues, causing the container `oom-test-container` to be terminated and restarted multiple times. This is evident 
from the events and pod status. The pod is in a CrashLoopBackOff state, indicating that it is continuously crashing
and being restarted.

Recommendations:

1. Review the application's memory usage and adjust the memory limit and request accordingly.
2. Investigate the cause of the OOM errors and optimize the application's memory usage.
3. Consider adding more resources (e.g., memory, CPU) to the node or cluster to prevent OOM errors.